In [ ]:
from dotenv import load_dotenv
load_dotenv()

### Compresión de contexto

Primero sin compresor

In [17]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + \
    d.page_content for i, d in enumerate(docs)]))

In [18]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("./directorio/Un mundo feliz - ALDOUS HUXLEY.txt", encoding="utf-8")
pages = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
retriever = FAISS.from_documents(texts, embeddings).as_retriever()
docs = retriever.invoke("El libro propone un sistema de gobierno totalitario?")
pretty_print_docs(docs)

Created a chunk of size 1441, which is longer than the specified 1000
Created a chunk of size 2613, which is longer than the specified 1000
Created a chunk of size 1342, which is longer than the specified 1000
Created a chunk of size 1175, which is longer than the specified 1000
Created a chunk of size 1054, which is longer than the specified 1000
Created a chunk of size 1032, which is longer than the specified 1000
Created a chunk of size 2362, which is longer than the specified 1000
Created a chunk of size 1138, which is longer than the specified 1000
Created a chunk of size 1176, which is longer than the specified 1000
Created a chunk of size 1307, which is longer than the specified 1000
Created a chunk of size 1678, which is longer than the specified 1000
Created a chunk of size 1764, which is longer than the specified 1000
Created a chunk of size 1695, which is longer than the specified 1000
Created a chunk of size 2488, which is longer than the specified 1000
Created a chunk of s

Document 1:

Desde luego, no hay razón alguna para que el nuevo totalita- 
rismo se parezca al antiguo. El Gobierno, por medio de porras 
y piquetes de ejecución, hambre artificialmente provocada, en- 
carcelamientos en masa y deportación también en masa no es 
solamente inhumano (a nadie, hoy día, le importa demasiado 
este hecho); se ha comprobado que es ineficaz, y en una época 
de tecnología avanzada la ineficacia es un pecado contra el Es- 


Un Mundo Feliz
----------------------------------------------------------------------------------------------------
Document 2:

La razón se confundió a sí misma 

al unirse la división... 

—iOrgía-Porfía! —gritó Bernard, interrumpiendo la lectura con 
una risa estruendosa, desagradable—. Parece exactamente un 
himno del Servicio de Solidaridad. 

Así se vengaba de sus dos amigos por el hecho de apreciarse más 
entre sí de lo que le apreciaban a él. 

Sin embargo, por extraño que pueda parecer, la siguiente in- 
terrupción, la más desafortun

Con compresor

In [19]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

template = """ Eres un estudiante de ciencias politicas.

Context: {context}

Question: {question}?
"""
prompt = PromptTemplate(input_variables=['question', 'context'],template = template)
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0, max_tokens=250)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                       base_retriever=retriever)
question = "El libro propone un sistema de gobierno totalitario?"
compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

Document 1:

El Gobierno, por medio de porras y piquetes de ejecución, hambre artificialmente provocada, encarcelamientos en masa y deportación también en masa...un sistema de gobierno totalitario...un Mundo Feliz

The government, through truncheons and execution squads, artificially created hunger, mass imprisonments and mass deportations...a totalitarian system...in a world called Happy.
----------------------------------------------------------------------------------------------------
Document 2:

The context does not provide any information that is relevant to answer the question. NO_OUTPUT.
----------------------------------------------------------------------------------------------------
Document 3:

NO_OUTPUT. The context does not provide information relevant to answer the question about the book proposing a totalitarian system of government.
----------------------------------------------------------------------------------------------------
Document 4:

NO_OUTPUT. The context

##### El siguiente codigo ejecuta el LLM con la informacion acumulada en compressed_docs
Pueden ustedes volver aqui luego de estudiar prompts

In [24]:
# Preparar el contexto uniendo los documentos en compressed_docs
context = "\n\n".join([doc.page_content for doc in compressed_docs])

# Crear una cadena LCEL
chain = prompt | llm

# Ejecutar la cadena
response = chain.invoke({
    "question": question,
    "context": context
})

# Imprimir la respuesta
print(response.content)

Based on the information you have provided, it seems that the book being referred to is set in a world where the government uses repressive measures such as truncheons, execution squads, artificially created hunger, mass imprisonments, and mass deportations to maintain control. These tactics are characteristic of a totalitarian system of government. However, I would need more information about the book to definitively say whether it proposes a totalitarian system of government or simply portrays one.

Therefore, the answer is: Maybe, based on the limited context provided.
